# MFCC Features for Masks

Simple example of linear regression being used in conjunction with MFCCs.

In [2]:
# Speech Libraries
from python_speech_features import mfcc, get_filterbanks
from python_speech_features.sigproc import deframesig
from psf_supplement import specdecomp

## Wave libraries
from IPython.display import Audio
from IPython.display import display
import scipy.io.wavfile as wav

## Standard python libraries
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline

## Mixing two signals together

In [3]:
audio_dir = "/data/fs4/datasets/magnolia/sisec/dev/"
fs, snda = wav.read(audio_dir+"dev_Ca1_Ce_A_src.wav")
fs, sndb = wav.read(audio_dir+"dev_Ca1_Ce_B_src.wav")
fs, sndc = wav.read(audio_dir+"dev_Ca1_Co_A_src.wav")


sndabc = snda+sndb+sndc

## Parameters

In [4]:
# Parameters
nfilt=64
numcep=64
nfft=512
winlen=0.01
winstep=0.005
ceplifter=0

## Feature extraction and spectral decomposition

In [6]:
# We can extract MFCC features from the combination of A & B signals
mfcc_feat = mfcc(sndabc,fs,nfilt=nfilt,numcep=numcep,nfft=nfft,
                 winlen=winlen,winstep=winstep,ceplifter=ceplifter,
                 appendEnergy=False)

# Since python_speech_features takes spectrograms differently than 
# you or I might, I've included the library in psf_supplement.py.
mfcc_magni = specdecomp(sndabc,samplerate=fs,nfft=nfft,
                        winlen=winlen,winstep=winstep,decomp='abs')
mfcc_phase = specdecomp(sndabc,samplerate=fs,nfft=nfft,
                        winlen=winlen,winstep=winstep,decomp='phase')

# Labeled mask requires situations where the desired signal is larger 
# than the noise. In this case, we require magnitude responses of signals
# A and B (the signal and noise).
magni_A = specdecomp(snda,samplerate=fs,nfft=nfft,
                     winlen=winlen,winstep=winstep,decomp='abs')
magni_B = specdecomp(sndb,samplerate=fs,nfft=nfft,
                     winlen=winlen,winstep=winstep,decomp='abs')
magni_C = specdecomp(sndc,samplerate=fs,nfft=nfft,
                     winlen=winlen,winstep=winstep,decomp='abs')


# Define the mask
mask_A = magni_A > (magni_B + magni_C)
mask_B = magni_B > (magni_A + magni_C)
mask_C = magni_C > (magni_B + magni_C)

## Deep Neural Network Solution

In [7]:
train_from_scratch = True
continue_to_train = True
# If you want to use a model: 'overfitted_dnn_mask.h5'
save_trained_model = 'overfitted_dnn_mask.h5' 
load_trained_model = None

import os
# os.environ['KERAS_BACKEND']= "tensorflow"
os.environ['KERAS_BACKEND']= "theano"
import keras
from keras.models import Sequential, load_model
from keras.optimizers import Adam
from keras.layers.core import Dense, Activation, Dropout, Lambda
from keras.objectives import binary_crossentropy

if train_from_scratch:
    model = Sequential()
    model.add(Dense(input_dim=mfcc_feat.shape[1], output_dim=2048, init="uniform"))
    model.add(Activation("relu"))
    model.add(Dense(input_dim=2048, output_dim=2048))
    model.add(Activation("relu"))
    model.add(Dense(input_dim=2048, output_dim=257))
    model.add(Activation("sigmoid"))
    optimizer_init=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    model.compile(loss=binary_crossentropy, optimizer=optimizer_init)

if continue_to_train:
    modelloss = model.fit(mfcc_feat, mask, nb_epoch=100, batch_size=32,verbose=2,shuffle=1)

if save_trained_model:
    model.save(save_trained_model, overwrite=False)
    
if load_trained_model:
    model = load_model(load_trained_model)
    
mask_recon = model.predict(mfcc_feat)

Using Theano backend.
Using gpu device 0: GeForce GTX TITAN X (CNMeM is disabled, cuDNN 5110)
/opt/conda/lib/python3.5/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


NameError: name 'mask' is not defined

## Recover FFT magnitude

First recover the magnitude response through power spectrum.

In [ ]:
snda_recon = (mask_recon * magni_A) * np.exp( 1j *  mfcc_phase )
snda_recon = np.fft.irfft( snda_recon )
snda_recon = snda_recon[:,:(int(fs*winlen))]

snda_recon = deframesig(snda_recon, 0, int(fs*winlen), int(fs*winstep))

In [ ]:
snda_recon = snda_recon.astype(np.int16)
plt.subplot(121); plt.imshow(np.log(mfcc_magni),aspect=0.4); 
plt.title('Original spectrum'); plt.colorbar(); plt.ylabel('Time (Sample)')
plt.subplot(122); plt.imshow(np.log(magni_A),aspect=0.4); 
plt.title('Reconstructed spectrum'); plt.colorbar(); plt.ylabel('Time (Sample)')
display(Audio(snda,rate=fs))
display(Audio(snda_recon,rate=fs))
